In [1]:
"""
LRU Cache Example and Tester.

Implements a simple limited size LRU cache class that stores items in a 
dictionary, as well as an ordered deque. New entries are added to the
front of the list, while the last entry is trimmed if an insert would
increase the size beyond the limit.

John Fogarty, john@jfogarty.org, https://github.com/jfogarty

"""
import sys

debugging = False 
#debugging = True
logging = True

MAX_CACHE = 1000000 # Arbitrary upper limit for this test example


In [2]:
#-----------------------------------------------------------------------------
# Console/Log outputs
def dbg(f, *args):
    if debugging:
        print(('  DBG:' + f).format(*args))
        

def log(f, *args):
    if logging:
        print((f).format(*args))
        
        
def log_error(f, *args):
    if logging:
        print(('*** ERROR:' + f).format(*args))     

In [4]:
#-----------------------------------------------------------------------------
class LruCacheDeque(object):
    """
    A Simple LRU Cache implented with dict and deque
    """
    def __init__(self, size):
        assert size > 1 and size <= MAX_CACHE
        self.vals  = {}
        self.size  = size       
        self.first = None
        self.last  = None
        dbg("- LruCacheDeque({0})", self.size)
        

    def remove(self, key):
        ''' Remove an arbitrary entry from the cache '''
        if key in self.vals:
            entry = self.vals[key]
            key  = entry['key']
            prev = entry['prev']
            next = entry['next']
            if self.first is entry:
                self.first = next
            if self.last is entry:
                self.last is prev
            del self.vals[key]
            
            
    def remove_oldest(self):
        ''' Remove the last entry from the deque ''' 
        entry = self.last
        if entry:
            key = entry['key']
            self.remove(key)
        
        
    def get(self, key):
        #dbg("Values: {0}", self.vals)
        try:
            return self.vals[key]['val']
        except KeyError:
            return None
        
        
    def insert(self, key, val):
        ''' Insert an item in the cache, making it the newest entry. '''
        if key in self.vals:       
            self.remove(key)
        if len(self.vals) == self.size:
            # 
            self.remove_oldest()
        entry = {'key':key, 'val':val, 'prev':None, 'next':self.first}
        self.first = entry
        self.vals[key] = entry
        if not self.last:
            self.last = entry

In [5]:
#-----------------------------------------------------------------------------
# Chatty responses
def say_error():
    log('ERROR')


def say_notfound():
    log('NOTFOUND')


def say_ok(kw):
    log(kw + ' OK')


def say_got(v):
    log('GOT {0}', v)  

In [6]:
#-----------------------------------------------------------------------------
def command_line(f, keywords):
    s = f.readline().strip()
    v = s.split()
    if len(v):
        cmd = v[0]
        if cmd in keywords:   
            return v
        say_error()
        dbg('Keyword must be in {0}', keywords)
    else:
        say_error()
        dbg('Missing keyword')

In [8]:
#-----------------------------------------------------------------------------
# Main test application
def main():
    dbg("- LRU Cache Example problem for BrandVerity 1.0")
    cls = LruCacheDeque

    with open('data/lruin.dat') as f:
        size = int(command_line(f, "SIZE")[1])
        assert size > 0 and size <= MAX_CACHE
        say_ok('SIZE')
        cache = cls(size)
            
        exit = False
        while not exit:
            v = command_line(f, ['EXIT', 'GET', 'SET'])
            kw = v[0]
            if kw == 'EXIT':
                exit = True
            else:
                dbg("- Command {0} : {1}", v[0], v[1:])
                key = v[1]
                if kw == 'SET':
                    if len(v) != 3:
                        say_error()
                    val = v[2]
                    cache.insert(key, val)
                    say_ok(kw)
                elif kw == 'GET':           
                    if len(v) != 2:
                        say_error()
                    else:
                        val = cache.get(key)
                        if val == None:
                            say_notfound()
                        else:
                            say_got(val)

    dbg('- Program exit')

In [9]:
main()

SIZE OK
NOTFOUND
SET OK
GOT 1
SET OK
GOT 1.1
SET OK
GOT 2
SET OK
SET OK
NOTFOUND
GOT 2
GOT third
ERROR
GOT four
